In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
import sklearn.metrics as mt
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from imblearn.combine import SMOTEENN
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


# record start time
start = time.time()
le = LabelEncoder()
# Importing the dataset
data = pd.read_csv('https://raw.githubusercontent.com/mahayasa/gan-hybrid-sampling-customer-churn/main/data/telco_new.csv')

X=data.drop(['Churn'],axis=1)
y=data["Churn"]
#y = le.fit_transform(y)
#imblearn SMOTE
#smote_enn = SMOTE()
smote_enn = SMOTEENN()
X, y = smote_enn.fit_resample(X, y)
#smote_enn = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
#X, y = smote_enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)

end = time.time()

print("The time of execution of preprocess:",
      (end-start), "s")

The time of execution of preprocess: 0.4890315532684326 s


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco3-dt-cs-auc.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco3-dt-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.977626 using {'class_weight': {0: 50, 1: 100}}
0.977517 (0.003931) with: {'class_weight': {0: 10, 1: 100}}
0.975252 (0.005696) with: {'class_weight': {0: 20, 1: 100}}
0.974356 (0.005197) with: {'class_weight': {0: 30, 1: 100}}
0.974283 (0.006346) with: {'class_weight': {0: 40, 1: 100}}
0.977626 (0.004827) with: {'class_weight': {0: 50, 1: 100}}
0.976016 (0.006506) with: {'class_weight': {0: 60, 1: 100}}
0.975566 (0.006658) with: {'class_weight': {0: 70, 1: 100}}
0.976329 (0.006173) with: {'class_weight': {0: 80, 1: 100}}
0.977213 (0.005980) with: {'class_weight': {0: 90, 1: 100}}
0.975153 (0.005402) with: {'class_weight': {0: 100, 1: 100}}


In [13]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco3-dt-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco3-dt-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.977389 using {'class_weight': {0: 80, 1: 100}}
0.976144 (0.004599) with: {'class_weight': {0: 10, 1: 100}}
0.975899 (0.005415) with: {'class_weight': {0: 20, 1: 100}}
0.975264 (0.004830) with: {'class_weight': {0: 30, 1: 100}}
0.975343 (0.005391) with: {'class_weight': {0: 40, 1: 100}}
0.976446 (0.004682) with: {'class_weight': {0: 50, 1: 100}}
0.976288 (0.007470) with: {'class_weight': {0: 60, 1: 100}}
0.975574 (0.005796) with: {'class_weight': {0: 70, 1: 100}}
0.977389 (0.006304) with: {'class_weight': {0: 80, 1: 100}}
0.976995 (0.005415) with: {'class_weight': {0: 90, 1: 100}}
0.975500 (0.005260) with: {'class_weight': {0: 100, 1: 100}}


In [14]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco3-dt-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco3-dt-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder


Best: 0.977163 using {'class_weight': {0: 10, 1: 100}}
0.977163 (0.004642) with: {'class_weight': {0: 10, 1: 100}}
0.976805 (0.004435) with: {'class_weight': {0: 20, 1: 100}}
0.975341 (0.005538) with: {'class_weight': {0: 30, 1: 100}}
0.976103 (0.004907) with: {'class_weight': {0: 40, 1: 100}}
0.975764 (0.006060) with: {'class_weight': {0: 50, 1: 100}}
0.976577 (0.007291) with: {'class_weight': {0: 60, 1: 100}}
0.974986 (0.005211) with: {'class_weight': {0: 70, 1: 100}}
0.976968 (0.005803) with: {'class_weight': {0: 80, 1: 100}}
0.976483 (0.006422) with: {'class_weight': {0: 90, 1: 100}}
0.975504 (0.004287) with: {'class_weight': {0: 100, 1: 100}}


In [15]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco3-lr-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco3-lr-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.926040 using {'class_weight': {0: 100, 1: 100}}
0.919777 (0.007481) with: {'class_weight': {0: 10, 1: 100}}
0.921479 (0.006903) with: {'class_weight': {0: 20, 1: 100}}
0.923175 (0.007178) with: {'class_weight': {0: 30, 1: 100}}
0.922426 (0.007401) with: {'class_weight': {0: 40, 1: 100}}
0.922695 (0.006994) with: {'class_weight': {0: 50, 1: 100}}
0.923843 (0.008315) with: {'class_weight': {0: 60, 1: 100}}
0.924053 (0.007013) with: {'class_weight': {0: 70, 1: 100}}
0.924546 (0.007211) with: {'class_weight': {0: 80, 1: 100}}
0.923259 (0.006716) with: {'class_weight': {0: 90, 1: 100}}
0.926040 (0.007672) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco3-lr-cs-f1.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco3-lr-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.853581 using {'class_weight': {0: 100, 1: 100}}
0.799501 (0.014182) with: {'class_weight': {0: 10, 1: 100}}
0.823908 (0.009877) with: {'class_weight': {0: 20, 1: 100}}
0.837819 (0.010243) with: {'class_weight': {0: 30, 1: 100}}
0.841038 (0.011351) with: {'class_weight': {0: 40, 1: 100}}
0.843962 (0.009880) with: {'class_weight': {0: 50, 1: 100}}
0.848361 (0.009717) with: {'class_weight': {0: 60, 1: 100}}
0.847833 (0.009246) with: {'class_weight': {0: 70, 1: 100}}
0.850479 (0.010465) with: {'class_weight': {0: 80, 1: 100}}
0.849251 (0.010063) with: {'class_weight': {0: 90, 1: 100}}
0.853581 (0.010913) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco3-lr-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco3-lr-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.851231 using {'class_weight': {0: 100, 1: 100}}
0.772090 (0.016797) with: {'class_weight': {0: 10, 1: 100}}
0.802030 (0.012151) with: {'class_weight': {0: 20, 1: 100}}
0.820197 (0.011760) with: {'class_weight': {0: 30, 1: 100}}
0.827383 (0.012778) with: {'class_weight': {0: 40, 1: 100}}
0.833083 (0.010068) with: {'class_weight': {0: 50, 1: 100}}
0.839834 (0.009553) with: {'class_weight': {0: 60, 1: 100}}
0.841719 (0.009485) with: {'class_weight': {0: 70, 1: 100}}
0.845789 (0.010712) with: {'class_weight': {0: 80, 1: 100}}
0.845577 (0.009833) with: {'class_weight': {0: 90, 1: 100}}
0.851231 (0.010854) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco3-svm-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco3-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.913287 using {'class_weight': {0: 80, 1: 100}}
0.780366 (0.017247) with: {'class_weight': {0: 10, 1: 100}}
0.817552 (0.017497) with: {'class_weight': {0: 20, 1: 100}}
0.855123 (0.018667) with: {'class_weight': {0: 30, 1: 100}}
0.891448 (0.016274) with: {'class_weight': {0: 40, 1: 100}}
0.907220 (0.011274) with: {'class_weight': {0: 50, 1: 100}}
0.911495 (0.009983) with: {'class_weight': {0: 60, 1: 100}}
0.913245 (0.009083) with: {'class_weight': {0: 70, 1: 100}}
0.913287 (0.008904) with: {'class_weight': {0: 80, 1: 100}}
0.903404 (0.012621) with: {'class_weight': {0: 90, 1: 100}}
0.896329 (0.012090) with: {'class_weight': {0: 100, 1: 100}}


In [19]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco3-svm-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco3-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.856520 using {'class_weight': {0: 100, 1: 100}}
0.790958 (0.011187) with: {'class_weight': {0: 10, 1: 100}}
0.797603 (0.010545) with: {'class_weight': {0: 20, 1: 100}}
0.804476 (0.010752) with: {'class_weight': {0: 30, 1: 100}}
0.807693 (0.010612) with: {'class_weight': {0: 40, 1: 100}}
0.810725 (0.009385) with: {'class_weight': {0: 50, 1: 100}}
0.811906 (0.009716) with: {'class_weight': {0: 60, 1: 100}}
0.812640 (0.009334) with: {'class_weight': {0: 70, 1: 100}}
0.813367 (0.009381) with: {'class_weight': {0: 80, 1: 100}}
0.829619 (0.013206) with: {'class_weight': {0: 90, 1: 100}}
0.856520 (0.010741) with: {'class_weight': {0: 100, 1: 100}}


In [20]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco3-svm-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco3-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.852302 using {'class_weight': {0: 100, 1: 100}}
0.761674 (0.013238) with: {'class_weight': {0: 10, 1: 100}}
0.769695 (0.012652) with: {'class_weight': {0: 20, 1: 100}}
0.778541 (0.013035) with: {'class_weight': {0: 30, 1: 100}}
0.782866 (0.012934) with: {'class_weight': {0: 40, 1: 100}}
0.786434 (0.011498) with: {'class_weight': {0: 50, 1: 100}}
0.787819 (0.011881) with: {'class_weight': {0: 60, 1: 100}}
0.788683 (0.011413) with: {'class_weight': {0: 70, 1: 100}}
0.789537 (0.011467) with: {'class_weight': {0: 80, 1: 100}}
0.815739 (0.018024) with: {'class_weight': {0: 90, 1: 100}}
0.852302 (0.011329) with: {'class_weight': {0: 100, 1: 100}}
